In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from BICS_ABM import BICS_ABM, VaccineRule
import time
from joblib import Parallel, delayed
from scipy.stats.qmc import LatinHypercube
from scipy.stats import norm, uniform, randint

import collections
import re

import datetime
import pickle
import dill

from scipy.signal import find_peaks

OSError: dlopen(/Users/eroubenoff/BICS_ABM/build/libBICS_ABM_lib.so, 0x0006): tried: '/Users/eroubenoff/BICS_ABM/build/libBICS_ABM_lib.so' (no such file)

# Analysis for new idea: Vaccine timing and outbreak cycles

Begin by developing treatment combinations

In [ ]:
dirname = "sims/2023-02-21/"
flist = [dirname + x for x in os.listdir(dirname) if '.pgz' in x]

In [ ]:
# Functions for analysis
from scipy.signal import find_peaks

def process_sim(f):
    # Parse beta0 and beta1 from file name
    beta = {s.split("_")[0]: float(s.split("_")[1]) for s in f.split("/")[2].split("__")[0:2]}
    with gzip.open(f, 'rb') as f:
        sim = pickle.load(f)
        
    vu = np.mean(sim._pop[:,[i for i,x in enumerate(sim._colnames) if x == "vaccine_priority" ][0]] == 0)
    
    ret = {**beta, 
            'T_REINFECTION': sim._params.T_REINFECTION,
              'BOOSTER_DAY': sim._params.BOOSTER_DAY,
              'VEBOOST': sim._params.VEBOOST,
              'VU': vu}
    N = sim.S[0] + sim.E[0]
    threshold = N * 0.01
    threshold= 2
    x = np.add.reduceat(sim.Cc, np.arange(0, len(sim.Cc), 24))
    peaks, properties = find_peaks(x, height=threshold, distance=30, width=1, rel_height=1)
    
    count = []
    for i in range(len(properties["left_ips"])):
        left_bound = round(properties["left_ips"][i])
        right_bound = round(properties["right_ips"][i])
        count.append(sum(x[left_bound:right_bound]))
            
    ret["num_outbreaks"] = len(peaks)
    ret["mean_time_bw_outbreaks"] = np.mean(np.diff(peaks))
    ret["avg_outbreak_size"] = np.mean(count)
    
    return ret
    

In [ ]:
processed_sims = []
for i, fname in enumerate(flist):
    print(i, end="\r")
    processed_sims.append(process_sim(fname))


In [ ]:
processed_sims = pd.DataFrame(processed_sims)

In [ ]:
fig, ax = plt.subplots(ncols=3,figsize= (10,5))
ax[0].plot(processed_sims.T_REINFECTION/24, processed_sims.num_outbreaks, ".", )
ax[0].set_title("Number of Outbreaks")
ax[0].set_xlabel("Time until reinfection (days)")
ax[1].plot(processed_sims.T_REINFECTION/24, processed_sims.mean_time_bw_outbreaks, ".")
ax[1].set_title("Avg. Time Between Outbreaks (days)")
ax[1].set_xlabel("Time until reinfection (days)")
ax[2].plot(processed_sims.T_REINFECTION/24, processed_sims.avg_outbreak_size, ".")
ax[2].set_title("Avg. Outbreak Size")
ax[2].set_xlabel("Time until reinfection (days)")

In [ ]:
processed_sims.groupby(["T_REINFECTION"]).agg(np.mean)

In [ ]:
fig, ax = plt.subplots(ncols=3,figsize= (10,5))
ax[0].plot(processed_sims.BOOSTER_DAY, processed_sims.num_outbreaks, ".", )
ax[0].set_title("Number of Outbreaks")
ax[0].set_xlabel("Booster Day")
ax[1].plot(processed_sims.BOOSTER_DAY, processed_sims.mean_time_bw_outbreaks, ".")
ax[1].set_title("Avg. Time Between Outbreaks (days)")
ax[1].set_xlabel("Booster Day")
ax[2].plot(processed_sims.BOOSTER_DAY, processed_sims.avg_outbreak_size, ".")
ax[2].set_title("Avg. Outbreak Size")
ax[2].set_xlabel("Booster Day")

In [ ]:
processed_sims.groupby(["BOOSTER_DAY"]).agg(np.mean)

In [ ]:
def closest_value(input_list, input_value):
 
  arr = np.asarray(input_list)
 
  i = (np.abs(arr - input_value)).argmin()
 
  return arr[i]

processed_sims["VU_rounded"] = [closest_value([0.25, 0.5, 0.75], x) for x in processed_sims.VU]

fig, ax = plt.subplots(ncols=3,figsize= (10,5))
ax[0].plot(processed_sims.VU_rounded, processed_sims.num_outbreaks, ".", )
ax[0].set_title("Number of Outbreaks")
ax[0].set_xlabel("Vaccine Uptake Rate")
ax[1].plot(processed_sims.VU_rounded, processed_sims.mean_time_bw_outbreaks, ".")
ax[1].set_title("Avg. Time Between Outbreaks (days)")
ax[1].set_xlabel("Vaccine Uptake Rate")
ax[2].plot(processed_sims.VU_rounded, processed_sims.avg_outbreak_size, ".")
ax[2].set_title("Avg. Outbreak Size")
ax[2].set_xlabel("Vaccine Uptake Rate")

In [ ]:
processed_sims.groupby(["VU_rounded"]).agg(np.mean)

# Calculating Average Treatment Effect

6 treatment variables: BETA0, BETA1, T_REINFECTION, VEBOOST, VU_rounded, BOOSTER_DAY

3 outcome variables: num_outbreaks, mean_time_bw_outbreaks, avg_outbreak_size


In [ ]:
processed_sims.groupby(["BETA0", "BETA1", "T_REINFECTION", "VEBOOST", "BOOSTER_DAY", "VU_rounded"]).agg(np.mean)

In [ ]:
np.mean(processed_sims[processed_sims.BETA1 > 0.5].avg_outbreak_size)
np.mean(processed_sims[processed_sims.BETA1 < 0.5].avg_outbreak_size)

## Booster Day

In [ ]:
# Let's start with one easy one. BOOSTER_DAY = march 1st (day 59) vs sept 1st (243)
booster_day_df = processed_sims.drop(
    "VU", axis=1
).pivot_table(
    index = ["BETA0", "BETA1", "T_REINFECTION", "VEBOOST", "VU_rounded"],
    columns = "BOOSTER_DAY",
    values = ["num_outbreaks", "mean_time_bw_outbreaks", "avg_outbreak_size"])
booster_day_df

In [ ]:
for i in range(1, 12):
    m1 = booster_day_df.avg_outbreak_size.columns[i]
    m0 = booster_day_df.avg_outbreak_size.columns[i-1]
    
    mte = np.mean(booster_day_df["avg_outbreak_size"][m1] - booster_day_df["avg_outbreak_size"][m0])
    mte = np.mean(booster_day_df["mean_time_bw_outbreaks"][m1] - booster_day_df["mean_time_bw_outbreaks"][m0])
    mte = np.mean(booster_day_df["num_outbreaks"][m1] - booster_day_df["num_outbreaks"][m0])
    
    print(mte)

## Vaccine uptake

In [ ]:
vu_df = processed_sims.drop(
    "VU", axis=1
).pivot_table(
    index = ["BETA0", "BETA1", "T_REINFECTION", "VEBOOST", "BOOSTER_DAY"],
    columns = "VU_rounded",
    values = ["num_outbreaks", "mean_time_bw_outbreaks", "avg_outbreak_size"])
vu_df

In [ ]:
fig, ax = plt.subplots(3, figsize=(5,10))

outcomes = ["avg_outbreak_size", "mean_time_bw_outbreaks", "num_outbreaks"]

for i, o in enumerate(outcomes):
    v25 = []
    v50 = []
    v75 = []
    b0_vec = vu_df[o].index.levels[0]
    for b0 in b0_vec:

        v25.append(np.mean(vu_df[o][0.25].loc[b0]))
        v50.append(np.mean(vu_df[o][0.50].loc[b0]))
        v75.append(np.mean(vu_df[o][0.75].loc[b0]))

    ax[i].plot(b0_vec, v25, "--go", label = "25% Vaccine Uptake")
    ax[i].plot(b0_vec, v50, "--bo", label = "50% Vaccine Uptake")
    ax[i].plot(b0_vec, v75, "--ro", label = "75% Vaccine Uptake")
    
    
ax[0].legend()
    
# ax[0].set_title("Higher Vaccine Uptake reduces outbreak size at all transmission probabilities")
ax[0].set_xlabel("Average Probability of Transmission")
ax[0].set_ylabel("Average Size of of Outbreaks")
ax[1].set_xlabel("Average Probability of Transmission")
ax[1].set_ylabel("Average Time Between Outbreaks (days)")
ax[2].set_xlabel("Average Probability of Transmission")
ax[2].set_ylabel("Average Number of Outbreaks")
#ax[0].set_ylabel("Average Clinical Infections per Outbreak")

plt.suptitle("Higher Vaccine Uptake reduces outbreak size, but increases frequency")
plt.show()


In [ ]:
fig, ax = plt.subplots()

v25 = []
v50 = []
v75 = []
b1_vec = vu_df["avg_outbreak_size"].index.levels[1]
for b1 in b1_vec:
    
    v25.append(np.mean(vu_df["avg_outbreak_size"][0.25].loc[:, b1]))
    v50.append(np.mean(vu_df["avg_outbreak_size"][0.50].loc[:, b1]))
    v75.append(np.mean(vu_df["avg_outbreak_size"][0.75].loc[:, b1]))
    
ax.plot(b1_vec, v25, "--go", label = "25% Vaccine Uptake")
ax.plot(b1_vec, v50, "--bo", label = "50% Vaccine Uptake")
ax.plot(b1_vec, v75, "--ro", label = "75% Vaccine Uptake")
ax.legend()
ax.set_title("Higher Vaccine Uptake reduces outbreak size at all forcing amplitudes")
ax.set_xlabel("Amplitude of Seasonal Forcing")
ax.set_ylabel("Average Clinical Infections per Outbreak")
    
plt.show()



# Month of Vaccination

In [ ]:
bd_df = processed_sims.drop(
    "VU", axis=1
).pivot_table(
    index = ["BETA0", "BETA1", "T_REINFECTION", "VEBOOST", "VU_rounded"],
    columns = "BOOSTER_DAY",
    values = ["num_outbreaks", "mean_time_bw_outbreaks", "avg_outbreak_size"])

fig, ax = plt.subplots(3, figsize=(5,10))

outcomes = ["avg_outbreak_size", "mean_time_bw_outbreaks", "num_outbreaks"]
months = bd_df["num_outbreaks"].columns

for i, o in enumerate(outcomes):
    df = pd.DataFrame(columns = months)
    b0_vec = bd_df[o].index.levels[0]
    for b0 in b0_vec:
        tmp = pd.DataFrame([{k: np.mean(bd_df[o][k].loc[b0]) for k in months}])
        df = pd.concat([df, tmp], ignore_index=True)

    for m in months:
        ax[i].plot(b0_vec, df.loc[:,m], "--o", label = m)
    
    
ax[0].legend()
    
ax[0].set_xlabel("Average Probability of Transmission")
ax[0].set_ylabel("Average Size of of Outbreaks")
ax[1].set_xlabel("Average Probability of Transmission")
ax[1].set_ylabel("Average Time Between Outbreaks (days)")
ax[2].set_xlabel("Average Probability of Transmission")
ax[2].set_ylabel("Average Number of Outbreaks")

plt.suptitle("No obvious effect of Month of Vaccination on outcomes across b0")
plt.show()


In [ ]:
bd_df = processed_sims.drop(
    "VU", axis=1
).pivot_table(
    index = ["BETA0", "BETA1", "T_REINFECTION", "VEBOOST", "VU_rounded"],
    columns = "BOOSTER_DAY",
    values = ["num_outbreaks", "mean_time_bw_outbreaks", "avg_outbreak_size"])

fig, ax = plt.subplots(3, figsize=(5,10))

outcomes = ["avg_outbreak_size", "mean_time_bw_outbreaks", "num_outbreaks"]
months = bd_df["num_outbreaks"].columns

for i, o in enumerate(outcomes):
    df = pd.DataFrame(columns = months)
    b1_vec = bd_df[o].index.levels[1]
    for b1 in b1_vec:
        tmp = pd.DataFrame([{k: np.mean(bd_df[o][k].loc[:,b1]) for k in months}])
        df = pd.concat([df, tmp], ignore_index=True)

    for m in months:
        ax[i].plot(b1_vec, df.loc[:,m], "--o", label = m)
    
    
ax[0].legend()
    
ax[0].set_xlabel("Amplitude of Seasonal Forcing")
ax[0].set_ylabel("Average Size of of Outbreaks")
ax[1].set_xlabel("Amplitude of Seasonal Forcing")
ax[1].set_ylabel("Average Time Between Outbreaks (days)")
ax[2].set_xlabel("Amplitude of Seasonal Forcing")
ax[2].set_ylabel("Average Number of Outbreaks")

plt.suptitle("No obvious effect of Month of Vaccination on outcomes across b0")
plt.show()


## Month of vaccination with t_reinf

In [ ]:
bd_df = processed_sims.drop(
    "VU", axis=1
).pivot_table(
    index = ["BETA0", "BETA1", "T_REINFECTION", "VEBOOST", "VU_rounded"],
    columns = "BOOSTER_DAY",
    values = ["num_outbreaks", "mean_time_bw_outbreaks", "avg_outbreak_size"])

fig, ax = plt.subplots(3, figsize=(5,10))

outcomes = ["avg_outbreak_size", "mean_time_bw_outbreaks", "num_outbreaks"]
months = bd_df["num_outbreaks"].columns

for i, o in enumerate(outcomes):
    df = pd.DataFrame(columns = months)
    t_vec = bd_df[o].index.levels[2]
    for t in t_vec:
        tmp = pd.DataFrame([{k: np.mean(bd_df[o][k].loc[:,:,t]) for k in months}])
        df = pd.concat([df, tmp], ignore_index=True)

    for m in months:
        ax[i].plot(t_vec, df.loc[:,m], "--o", label = m)
    
    
ax[0].legend()
    
ax[0].set_xlabel("Amplitude of Seasonal Forcing")
ax[0].set_ylabel("Average Size of of Outbreaks")
ax[1].set_xlabel("Amplitude of Seasonal Forcing")
ax[1].set_ylabel("Average Time Between Outbreaks (days)")
ax[2].set_xlabel("Amplitude of Seasonal Forcing")
ax[2].set_ylabel("Average Number of Outbreaks")

plt.suptitle("No obvious effect of Month of Vaccination on outcomes across b0")
plt.show()


# How does transmission affect?

In [ ]:
import seaborn as sns

In [ ]:
# To do this, group simulations by transmission parameters

# processed_sims.groupby(["BETA0", "BETA1"])
arr = processed_sims[processed_sims.BOOSTER_DAY == 31][["BETA0", "BETA1", "avg_outbreak_size"]]
print(arr)
#arr.pivot("BETA0", "BETA1", "avg_outbreak_size")
#sns.heatmap(data=arr, x="BETA0", y="BETA1", hue="avg_outbreak_size")

In [ ]:
import statsmodels.api as sm

In [ ]:
model = sm.OLS(processed_sims["avg_outbreak_size"],
               processed_sims[["BETA0", "BETA1", "BOOSTER_DAY","T_REINFECTION", "VEBOOST", "VU_rounded"]]).fit()

print(model.summary())

model = sm.OLS(processed_sims["mean_time_bw_outbreaks"],
               processed_sims[["BETA0", "BETA1", "BOOSTER_DAY","T_REINFECTION", "VEBOOST", "VU_rounded"]]).fit()

print(model.summary())

model = sm.OLS(processed_sims["num_outbreaks"],
               processed_sims[["BETA0", "BETA1", "BOOSTER_DAY","T_REINFECTION", "VEBOOST", "VU_rounded"]]).fit()

print(model.summary())

In [ ]:
booster_day_df["avg_outbreak_size"].loc[:,0,:,:]